# Myllia - echoes of silenced genes
---

**authors**: [fsb2210](https://www.kaggle.com/fsb2210), [julianc93](https://www.kaggle.com/julianc93)

The task is to train a model that is able to predict *expression changes in scRNA-seq data induced by CRISPRi perturbations*. For that, we have a dataset of 80 different perturbations and the *average expression values* of genes, plus an unperturbed case (*non-targeting sgRNA*).

## Introduction

This problem esentially consists of inputs of strings given by the `pert_symbol` column and an output vector space of dimension equal to the number of columns minus the one corresponding to the `pert_symbol` (i.e., a 5127-dimensional space).

Clearly, this dataset needs to be preprocessed. In particular, we are going to:

- treat each of the genes in the sample as tokens and get embeddings from them using a pre-trained neural network,
- compute delta averages for each of the output vectors,
- train a machine learning (ML) model using the preprocessed datasets created during steps 1 and 2,
- predict delta averages on new genes using the validation set (`pert_ids_val.csv`).

> **NOTE**
> 
> Dependencies list is: matplotlib numpy pandas tqdm scipy scikit-learn seabon

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.sparse as sp
import seaborn as sns
from tqdm import tqdm
import time
import warnings
warnings.filterwarnings("ignore")

from sklearn.decomposition import PCA
from sklearn.linear_model import RidgeCV, MultiTaskElasticNetCV
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import StandardScaler

In [ ]:
import sys
sys.path.append("/kaggle/usr/lib/relative-wmae-multiplied-by-wcosine")

from metric import _score_impl

Define global options:

In [ ]:
experiment_name = ""

# directory with data files
data_dir = "../data"
working_dir = "../data"

# random state integer value for reproducibility concerns
random_state = 42

# threshold value to consider PPI interactions ONLY if score is larger than this
score_threshold = 0.5

# restart probability of continuing the perturbation in the network. if this is an array, compute every one of them and concatenate results
alpha_RW = 0.75

# variance percentile to keep genes that vary significantly across perturbations, used for training a ML model
reduce_by_variance = False
variance_percentile = 80  # keep top 20%

# reduce dimensions of output space genes to this number in the latent space
n_latent = 35

# filename of figure
fig_fname = f"{experiment_name}.png"

# OOF predictions and submission filenames (DataFrame structures saved)
oof_preds_fname = f"{experiment_name}.csv"
submission_fname = f"submissions_{experiment_name}.csv"

Load datasets:

In [ ]:
# !curl -o links.txt.gz https://stringdb-downloads.org/download/protein.links.v12.0/9606.protein.links.v12.0.txt.gz
# !curl -o info.txt.gz https://stringdb-downloads.org/download/protein.info.v12.0/9606.protein.info.v12.0.txt.gz
# ! gunzip links.txt.gz info.txt.gz

In [ ]:
# links between proteins
links_df = pd.read_csv(f"{working_dir}/links.txt", sep=" ")
# protein name -> stringID
info_df = pd.read_csv(f"{working_dir}/info.txt", sep="\t")

In [ ]:
# train & validation sets
train_df = pd.read_csv(f"{data_dir}/training_data_means.csv")
val_df = pd.read_csv(f"{data_dir}/pert_ids_val.csv")
# ground truth
ground_truth = pd.read_csv(f"{data_dir}/training_data_ground_truth_table.csv")

Prepare gene lists:

In [ ]:
pert_df = train_df[train_df.pert_symbol != "non-targeting"]
cntrl_df = train_df[train_df.pert_symbol == "non-targeting"]

# gene inputs (labels)
train_pert_genes = pert_df["pert_symbol"].unique().tolist()
val_pert_genes = val_df["pert"].tolist()
all_pert_genes = train_pert_genes + val_pert_genes

# gene outputs (labels)
output_genes = [c for c in pert_df.columns if c != "pert_symbol"]

core_genes = list(set(all_pert_genes + output_genes))
print(f"- total number of genes in the training + validation sets (a.k.a., core_genes): {len(core_genes)}")

Load the ground truth values matching the training set. These values will be used during model training, in order to find the best solution (maximizing the metric):

In [ ]:
weight_cols = [f"w_{g}" for g in output_genes]
# true values (used later on, when computing metric)
Y_true = ground_truth[output_genes].values
W_true = ground_truth[weight_cols].values
baseline_true = ground_truth["baseline_wmae"].values

Create a dictionary mapping string ids from proteins to names. E.g., "9606.ENSP00000000233" -> "ARF5"

In [ ]:
string_to_name = dict(zip(info_df["#string_protein_id"], info_df["preferred_name"]))

Now we can add protein names to links DataFrame:

In [ ]:
# remove unnecesary columns & compute weights between [0, 1]
edges_df = links_df.copy()
edges_df["weight"] = edges_df["combined_score"] / 1000.0
edges_df = edges_df.drop(columns=["combined_score"])

edges_df["protein1"] = edges_df["protein1"].map(string_to_name)
edges_df["protein2"] = edges_df["protein2"].map(string_to_name)
edges_df = edges_df.dropna(subset=["protein1", "protein2"])  # remove unmapped edges

Just use links that have a score above a threshold:

In [ ]:
edges = edges_df[edges_df["weight"] >= score_threshold]

Find all interactions where at least one side of the proteins is in the core list (`core_genes`):

In [ ]:
mask = edges.protein1.isin(core_genes) | edges.protein2.isin(core_genes)
relevant_edges = edges[mask]
relevant_edges.shape

Create map between genes and index. First for the connected genes (cases with interactions) then for the ghost ones (no interactions):

In [ ]:
connected_genes = set(relevant_edges.protein1.unique()) | set(relevant_edges.protein2.unique())

# genes with edges
final_gene_set = connected_genes.copy()

# genes missing
missing_core = set(core_genes) - connected_genes

len(connected_genes), len(missing_core)

In [ ]:
# upda final list of genes with connected & missing ones
final_gene_set.update(missing_core)

# mapping between gene names and indices
gene_to_idx = {gene: idx for idx, gene in enumerate(final_gene_set)}

# and we can define the number of nodes!
n_nodes = len(gene_to_idx)

In [ ]:
print(f"- number of connected genes: {len(connected_genes)}")
print(f"- number of genes missing in STRING database: {len(missing_core)}")

We can now create the graph!

In [ ]:
src = relevant_edges["protein1"].map(gene_to_idx).values
dst = relevant_edges["protein2"].map(gene_to_idx).values
w = relevant_edges["weight"].values

rows = np.concatenate([src, dst])
cols = np.concatenate([dst, src])
data = np.concatenate([w, w])

Create the adjacency matrix along with the degree matrix (both being sparse matrices):

In [ ]:
# weighted adjancency matrix
A = sp.csr_matrix((data, (rows, cols)), shape=(n_nodes, n_nodes))

# add a small self-loop to prevent a division by zero when computing D^-1.
eye = sp.eye(n_nodes, format="csr")
A = A + eye

# degree matrix
deg = np.array(A.sum(axis=1)).flatten()

In [ ]:
num_iso = np.sum(deg == 0)
num_con = n_nodes - num_iso
print(f"- final graph nodes: {n_nodes}")
print(f"- minimum degree: {deg.min():.4f}")

Next, we create the random-walk Laplacian matrix, defined as:

\begin{equation}
    L = I - A D^{-1}
\end{equation}

where $I$ is the identity matrix, $D^{-1}$ is the inverse of the degree matrix and $A$ is the adjacency matrix.

This Laplacian matrix is the kernel of the heat equation (see below).

In [ ]:
# inverse matrix of degree
deg_inv = np.zeros_like(deg)
mask = deg > 0
deg_inv[mask] = 1 / deg[mask]
D_inv = sp.diags(deg_inv)

# Laplacian matrix
## L = sp.eye(n_nodes) - A.dot(D_inv)

### Diffusion operator

Now we are ready to create the operator which is needed to solve diffusion for diffferent perturbations.

The operator is defined as:

\begin{equation}
    x = (I + \epsilon L) x_0
\end{equation}

where $x$ is the solution we seek, that is, it is the perturbation vector integrated over time (steady-state). $x_0$ is the initial perturbation vector which is a one-hot vector localized at the gene that is perturbed. This is defined for a single "timestep".

Thus, the integrated solution is found to be:

\begin{equation}
    x = (1 - \alpha) (I - \alpha  A D^{-1})^{-1} x_0
\end{equation}

where $\alpha$ is a control parameter for the restart of the perturbation to its initial location in the graph.

Create auxiliary functions:

1. to one-hot encode vectors,
2. to solve the diffusion equation (diffusion operator) using the **conjugate gradient** method.

In [ ]:
def one_hot(gene, gene_to_idx):
    x0 = np.zeros(len(gene_to_idx))
    x0[gene_to_idx[gene]] = 1.0
    return x0

def diffuse_gene(gene, gene_to_idx, LU):
    x0 = one_hot(gene, gene_to_idx)
    x = LU.solve(x0)
    return x

Given the nature of the matrix, we can compute its LU decomposition using scipy.sparse.linalg method `splu`:

In [ ]:
start_time = time.time()

if isinstance(alpha_RW, list):
    all_X_train, all_X_val = [], []
    for alpha in alpha_RW:
        # M operator
        M = (1/(1 - alpha_RW)) * (sp.eye(n_nodes) - alpha_RW * A.dot(D_inv))
        M_csc = M.tocsc()
        # LU decomposition
        LU = sp.linalg.splu(M_csc)
        # diffuse on training & valid sets
        X_alpha_train = np.vstack([diffuse_gene(g, gene_to_idx, LU) for g in train_pert_genes])
        X_alpha_val = np.vstack([diffuse_gene(g, gene_to_idx, LU) for g in val_pert_genes])

        all_X_train.append(X_alpha_train)
        all_X_val.append(X_alpha_val)

    # stack diffusions
    X = np.hstack(all_X_train)
    X_val = np.hstack(all_X_val)

else:
    # M operator
    M = (1/(1 - alpha_RW)) * (sp.eye(n_nodes) - alpha_RW * A.dot(D_inv))
    M_csc = M.tocsc()
    # LU decomposition
    LU = sp.linalg.splu(M_csc)
    # stack diffusions
    X = np.vstack([diffuse_gene(g, gene_to_idx, LU) for g in train_pert_genes])
    X_val = np.vstack([diffuse_gene(g, gene_to_idx, LU) for g in val_pert_genes])

training_time = time.time() - start_time
print(f"LU decomposition completed in {training_time:.2f} seconds ({training_time/60:.2f} minutes)")

Example usage for a single gene "ACLY":

In [ ]:
"""
gene_example = "ACLY"

x0 = one_hot(gene_example, gene_to_idx)
idx = gene_to_idx[gene_example]

x = LU.solve(x0)
r = M @ x - x0
res = np.linalg.norm(r)

print(f"perturb. around inits: {x[idx-5:idx+5]}")
print(f"residual (r = (M @ x) - x0): {res:.4e}")
"""

Finally, we can create our input and output (`X`, `Y`) datasets to be used for training a machine learning model.

- `X` will be the full diffusion embedding matrix for each of the training perturbations,
- `Y` is the delta expressions of the 5127 genes of the training set.

Same idea for the `X_val` and `Y_val` for the validation set.

In [ ]:
# reduce dimensions of inputs to genes that actually have a non-negligible variance across perturbations (CHANGE 1A)
if reduce_by_variance:
    gene_var = np.var(X, axis=0)
    var_threshold = np.percentile(gene_var, variance_percentile)
    selected_mask = gene_var >= var_threshold
    top_ind = np.where(selected_mask)[0]
    X = X[:, top_ind]
    X_val = X_val[:, top_ind]

# create array of delta averages
Y_raw = pert_df.iloc[:,1:].values
Y_control = cntrl_df.iloc[:,1:].values
Y = Y_raw - Y_control

pca_resp = PCA(n_components=n_latent, random_state=random_state)
Y = pca_resp.fit_transform(Y)

X.shape, Y.shape, X_val.shape

## Model training

We are now ready to train a machine learning model using the results of the diffusion of perturbations in the graph as inputs and the computed delta expressions for 5127 genes as outputs.

In order to accomplish this task, we use a nested cross-validation (CV) technique containing two loops:

1. an inner CV that is used to tune the best hyperparameters of the model. takes samples of 79 perturbations, splits it into different folds to test different parameters and chooses the best ones,
2. an outer loop (**leave-one-out**, LOO) that is used to evaluate the best trained models (from the inner loop) and it generates a prediction for *every sample* in the set without that sample ever being used to train the final model that predicted it.

This is required for small datasets to avoid data leakages that would give rise to a very biased score.

Thus, the workflow for a single step is:

1. *split*: take sample 1 out (`va`), keep samples 2-80 (`tr`),
2. *scale*: fit scaler on samples 2-80, then transform samples 2-80 and sample 1,
3. *inner CV (tuning)*:
   - `RidgeCV` takes samples 2-80,
   - splits samples 2-80 into 3 chunks,
   - tests different alphas to see which one works best on the chunks,
   - picks  best parameters (e.g., `alpha` = 0.5)
4. *final train*: `RidgeCV` retrains on all of samples 2-80 using `alpha` = 0.5,
5. *out-of-fold (OOF) predict*: `RidgeCV` from step before makes a prediction on sample 1.

In [ ]:
start_time = time.time()

loo = LeaveOneOut()

# alpha values to try for the ElasticNet model
alphas_enet = [0.01, 0.1, 1, 100]
# alpha values to try in the Ridge model
alphas_r = [0.01, 0.1, 1, 100]

oof_preds = np.zeros((len(X), len(output_genes)))
val_preds = np.zeros((len(X_val), len(output_genes)))
scores, mapes = [], []
pbar = tqdm(enumerate(loo.split(X), 1), desc="model training", total=len(X), initial=1)
for k, (tr, va) in pbar:
    X_tr, X_va = X[tr], X[va]
    Y_tr, Y_va = Y[tr], Y[va]
    W_va = W_true[va]
    B_va = baseline_true[va]

    # scale features
    scaler = StandardScaler()
    X_tr_scaled = scaler.fit_transform(X_tr)
    X_va_scaled = scaler.transform(X_va)
    X_val_scaled = scaler.transform(X_val)

    # === Ridge model
    model_r = RidgeCV(alphas=alphas_r, cv=3)
    model_r.fit(X_tr_scaled, Y_tr)
    Y_pred_r = model_r.predict(X_va_scaled)
    Y_val_r = model_r.predict(X_val_scaled)

    # === ElasticNet model
    model_enet = MultiTaskElasticNetCV(l1_ratio=0.5, alphas=alphas_enet, cv=3, random_state=random_state, n_jobs=-1, max_iter=1000)
    model_enet.fit(X_tr_scaled, Y_tr)
    Y_pred_enet = model_enet.predict(X_va_scaled)
    Y_val_enet = model_enet.predict(X_val_scaled)

    # average predictions
    Y_pred = (Y_pred_enet + Y_pred_r) / 2
    Y_pred = pca_resp.inverse_transform(Y_pred)
    Y_va = pca_resp.inverse_transform(Y_va)

    # save OOF predictions
    oof_preds[va] = Y_pred.copy()

    # make predictions on validation set, final answer is average over the entire models
    Y_val_pred = (Y_val_enet + Y_val_r) / 2
    val_fold_preds = pca_resp.inverse_transform(Y_val_pred)
    val_preds += val_fold_preds / len(X)

    # score of OOF prediction (comparison against ground truth)
    fold_mape = mean_absolute_percentage_error(Y_va, Y_pred)
    fold_score = _score_impl(Y_va, Y_pred, W_va, B_va, eps=1e-12, max_log2=5.0, cos_left=0.0, cos_right=0.2)
    scores.append(fold_score)
    mapes.append(fold_mape)

    pbar.set_postfix({"iter": f"{k}/{len(X)}", "score": f"{fold_score:.4f}", "mape": f"{fold_mape:.4f}"})

training_time = time.time() - start_time
print(f"training completed in {training_time:.2f} seconds ({training_time/60:.2f} minutes)")
print(f"- OOF mean score: {np.mean(scores):.4f} (MAPE: {np.mean(mapes):.4f})")

In [ ]:
# calculate correlation per gene (axis=0)
y_true = pca_resp.inverse_transform(Y)
corrs = []
for i in range(y_true.shape[1]):
    # handle cases where variance is 0 to avoid NaNs
    if np.std(y_true[:, i]) > 1e-6:
        corr = np.corrcoef(y_true[:, i], oof_preds[:, i])[0, 1]
        corrs.append(corr)
corrs = np.array(corrs)

In [ ]:
# select top 50 most variable genes based on ground truth variance
gene_vars = np.var(y_true, axis=0)
top_idx = np.argsort(gene_vars)[-100:]
y_true_subset = y_true[:, top_idx]
pred_subset = oof_preds[:, top_idx]

fig_shape = (16, 9)
fig = plt.figure(figsize=fig_shape)

# Top row: two heatmaps with shared colorbar
ax1 = plt.subplot2grid((20, 20), (0, 0), colspan=8, rowspan=10)
im1 = sns.heatmap(y_true_subset, cmap="coolwarm", center=0, ax=ax1, 
                  cbar=False, vmin=-2, vmax=2)
ax1.set_title("Ground truth")
ax1.set_ylabel("pert. index")

ax2 = plt.subplot2grid((20, 20), (0, 10), colspan=9, rowspan=10)
im2 = sns.heatmap(pred_subset, cmap="coolwarm", center=0, ax=ax2, 
                  cbar=True, vmin=-2, vmax=2)
ax2.set_title("Prediction")
ax2.set_ylabel("pert. index")

# Bottom row: correlation histogram spanning full width
ax3 = plt.subplot2grid((20, 20), (12, 0), colspan=18, rowspan=9)
sns.histplot(corrs, bins=50, kde=True, ax=ax3)
ax3.axvline(x=0, color="black", linestyle=":", lw=3)
ax3.set_xlabel("Pearson correlation per gene")
ax3.set_ylabel("count of genes")

plt.savefig(fig_fname);

In addition, we also save OOF predictions in case we want to use them for an ensemble stacking or hill climbing approach:

In [ ]:
oof_preds_df = pd.DataFrame(oof_preds, columns=output_genes)
oof_preds_df.insert(0, "pert_symbol", train_pert_genes)
oof_preds_df.to_csv(oof_preds_fname, index=False)

## Inference

We are now ready to save predictions on the 60 genes found in the validation dataset along with 60 more dummy predictions for the not-yet available genes in the test dataset:

In [ ]:
val_ids = val_df["pert_id"].tolist()

preds = pd.DataFrame(val_preds, columns=output_genes)
preds.insert(0, "pert_id", val_ids)
pad = pd.DataFrame(0, index=range(60), columns=output_genes)
pad.insert(0, "pert_id", [f"pert_{i}" for i in range(61, 121)])

submission = pd.concat([preds, pad], ignore_index=True)
submission.to_csv(submission_fname, index=False)
submission.head()

---

In [ ]:
!rm -f info.txt links.txt